$$ \newcommand{\ket}[1]{\left|{#1}\right\rangle} $$
$$ \newcommand{\bra}[1]{\left\langle{#1}\right|} $$
$$ \newcommand{\expval}[1]{\left\langle{#1}\right\rangle} $$

# Variational Quantum Eigensolver
BLABLA introduction

### Motivation
Quantum Phase Estimation (QPE) is a common method in quantum computation to estimate eigenvalues of some unitary operator corresponding to an eigenvector. It has been shown to provide an exponential speed-up over current classical algorithms for determining ground state energies of quantum systems. However, to get good accuracy a long coherent evolution has to be performed, and as engineers struggle to achieve acceptable coherence times we can look for other methods better suited for near-term quantum computers. 

The Variational Quantum Eigensolver (VQE), unlike QPE, requires only a short coherent evolution. It is a hybrid algorithm utilizing both quantum and classical computers. In other words, this algorithm is specifically aimed at near-term quantum computers, and only requires state preparation and measurement of Pauli operators which is done in polynomial time. Can implement wave function ansätzes that are exponentially costly on classical computers. VQE is resistant to gate noise in some cases

### Algorithm
Assume we have an initial state of qubits $\ket{\Phi}$, we want to prepare our trial ground-state stationary wave function $\ket{\Psi(\theta)}$, dependent on some set of variables $\theta$, by an action of a suitable unitary operator $\hat{U}(\theta)$.
\begin{equation}
    \ket{\Psi} = \hat{U}\ket{\Phi}
\end{equation}
By variationally optimizing $\hat{U}$ we can find estimates to the ground state energy
\begin{equation}
    \frac{\bra{\Psi}H \ket{\Psi}}{\bra{\Psi}\ket{\Psi}} \leq E_0
\end{equation}
This is commonly known as the variational method. To implement this, we divide our algorithm into two subroutines; one quantum and one classical. The quantum subroutine prepares our state followed by multiple measurements of the expectation value in an appropriate basis. The classical subroutine will optimize the expectation value, with gradient descent or such, by varying the parameters. The process is then repeated.


## Example:
# Heisenberg model
The Heiseinberg model is a one-dimensional chain of spin $\frac{1}{2}$ particles, where there are only one degree of freedom being the spin. The direction of the spins and the relationship between neighbouring spins determine the energy of the system. The Hamiltonian is given by

$$
    H = h \sum_k \sigma_k^z \pm J\sum_k \vec{\sigma_k} \cdot \vec{\sigma_{k+1}}
$$
We write this in terms of dimensionless variables, by dividing with $\pm J$

$$
    \frac{H'}{\pm J} = h' \sum_k \sigma_k^z + \sum_k \vec{\sigma_k} \cdot \vec{\sigma_{k+1}}
$$

with $h'=\frac{h}{\pm J}$. 
Next we write out $\sigma$ vector dot products as

$$
\vec{\sigma_k} \cdot \vec{\sigma_{k+1}} = \sigma_k^x\sigma_{k+1}^x + \sigma_k^y\sigma_{k+1}^y + \sigma_k^z\sigma_{k+1}^z
$$

We are then left with the following expectation values
\begin{align}
    \expval{H'} = h'\sum_k \expval{\sigma_k^z} + \sum_k \left[ \expval{\sigma_k^x\sigma_{k+1}^x} + \expval{\sigma_k^y\sigma_{k+1}^y} + \expval{\sigma_k^z\sigma_{k+1}^z} \right]
\end{align}

In [25]:
import numpy as np
import qiskit as qk
from scipy.optimize import minimize

class Heisenberg:
    def __init__(self,particles,h):
        self.n = particles
        self.expvals = 4
        self.parameters = [h,1,1,1]
        self.shots = 1024
        
    def ansatz(self,qc,qb,theta):
        for i in range(self.n):
            #qc.ry(theta[i],qb[i])
            qc.h(qb[i])
            qc.rz(theta[i],qb[i])
        for i in range(self.n-1):
            qc.cx(qb[i],qb[i+1])
        qc.cx(qb[-1],qb[0])
    
    def expval(self,qc,qb,i,k):
        if k == self.n-1 and i != 0:
            k = -1
        if i == 0:
            # Onebody
            qc.z(qb[k])
        
        if i == 1:
            # Twobody X
            qc.x(qb[k])
            qc.x(qb[k+1])
            
        if i == 2:
            # Twobody Y
            qc.y(qb[k])
            qc.y(qb[k+1])
            
        if i == 3:
            # Twobody Z   
            qc.z(qb[k])
            qc.z(qb[k+1])
            
    def measure(self,qc,qb,cb,i,k):
        if k == self.n-1 and i != 0:
            k = -1
        
        if i == 1:
            # Twobody X
            qc.h(qb[k])
            qc.h(qb[k+1])
            
        if i == 2:
            # Twobody Y
            qc.h(qb[k])
            qc.h(qb[k+1])
            
        if i == 3:
            # Twobody Z
            qc.cx(qb[k+1],qb[k])
        
        qc.measure(qb,cb)
        job = qk.execute(qc,backend=qk.Aer.get_backend('qasm_simulator'),shots=self.shots)
        res = job.result().get_counts()
        E = 0
        for key,hits in res.items():
            eigenstate = key[::-1]
            if i == 0:
                q1 = 1 if eigenstate[k] == 0 else -1
                E += q1*hits
            else:
                q1 = 1 if eigenstate[k] == 0 else -1
                q2 = 1 if eigenstate[k+1] == 0 else -1
                eigenvalue = q1*q2
                E += eigenvalue*hits
        E /= self.shots
        return E*self.parameters[i]

    def ExpectationValue(self,theta):
        H = 0
        for k in range(self.n):
            for i in range(self.expvals):
                qb = qk.QuantumRegister(self.n)
                cb = qk.ClassicalRegister(self.n)
                qc = qk.QuantumCircuit(qb,cb)

                # ansatz
                self.ansatz(qc,qb,theta)

                # pauli word
                self.expval(qc,qb,i,k)

                # measurement
                H += self.measure(qc,qb,cb,i,k)

        return -1*H
        
    
def VariationalQuantumEigensolver(model,method='L-BFGS-B',tol=1e-10):
    theta = 2*np.pi*np.random.randn(model.n)
    if method == 'L-BFGS-B':
        bounds = [(0,2*np.pi) for i in theta]
    else:
        bounds = None
    result = minimize(model.ExpectationValue,theta,bounds=bounds,method=method,options={'disp':True},tol=tol)
    expectation = result.x
    print(ExpectationValue(expectation,model))
    return expectation

model = Heisenberg(2,1)
e = VariationalQuantumEigensolver(model)
print(e)

4.0
[0. 0.]
